# PyCity Schools Analysis

- At first glance, students at charter schools appear to be performing better overall than students at public schools. Of the fifteen schools provided, the charter schools are the highest performing by percentage of students passing overall, while the public schools are the lowest performing by percentage of students passing overall. After closer inspection, however, we observe that only one out of the seven charter schools represented in this dataset are considered large, while all eight public schools are considered large. This is significant because the sample size of the public schools is larger than the charter school, resulting in greater variability. It's important to be congnizant of other factors that could be contributing to the trends observed here as well, such as geographic location, and that numbers only tell part of the story.
  
---

In [131]:
#Import the pandas function library
import pandas as pd

#Open and read the csv files
school_data = pd.read_csv('Resources/schools_complete.csv')
student_data = pd.read_csv('Resources/students_complete.csv')

#Combine the csv files
school_and_student = pd.merge(student_data, school_data, how='left', on=['school_name', 'school_name'])

## District Summary

In [96]:
#Count the number of occurences each unique school appears in 'school_name'
school_count = school_and_student['school_name'].nunique()

In [97]:
#Count the number of rows in the 'student_name' column
student_count = school_and_student['student_name'].count()

In [98]:
#Add the budget of every unique school together
total_budget = school_and_student.groupby('school_name').budget.max().sum()

In [99]:
#Add the values in the 'math_score' column
math_sum = school_and_student['math_score'].sum()

#Calculate the average math score
average_math_score = math_sum/student_count

In [100]:
#Add the values in the 'reading_score' column
reading_sum = school_and_student['reading_score'].sum()

#Calculate the average reading score
average_reading_score = reading_sum/student_count

In [101]:
#Add math scores above 70 for 'student_name' column only
passing_math_count = school_and_student[(school_and_student['math_score'] >= 70)].count()['student_name']

#Calculate, on average, how many students are passing math
passing_math_percentage = passing_math_count / float(student_count) * 100

In [102]:
#Add reading scores above 70 for 'student_name' column only
passing_reading_count = school_and_student[(school_and_student['reading_score'] >= 70)].count()['student_name']

#Calculate, on average, how many students are passing reading
passing_reading_percentage = passing_reading_count / float(student_count) * 100

In [103]:
#Determine the number of students passing both math and reading
passing_math_reading_count = school_and_student[
    (school_and_student['math_score'] >= 70) & (school_and_student['reading_score'] >= 70)
].count()['student_name']

#Calculate, on average, how many students are passing both math and reading
overall_passing_rate = passing_math_reading_count /  float(student_count) * 100

In [104]:
#Summarize above calculations for all schools in new DataFrame
district_summary = pd.DataFrame({
    'Total Schools':school_count,
    'Total Students': student_count,
    'Total Budget': total_budget,
    'Average Math Score': average_math_score,
    'Average Reading Score': average_reading_score,
    'Students Passing Math (Percent)': passing_math_percentage,
    'Students Passing Reading (Percent)': passing_reading_percentage,
    'Students Passing Both (Percent)': overall_passing_rate
},index=[0])

#Formatting
district_summary['Total Students'] = district_summary['Total Students'].map("{:,}".format)
district_summary['Total Budget'] = district_summary['Total Budget'].map("${:,.2f}".format)

## School Summary

In [105]:
#Group calculations by school
school_summary = school_and_student.groupby('school_name')

In [106]:
#Display school type for each school
school_type = school_summary.type.max()

In [107]:
#Group student count by school
school_groupby = school_and_student.groupby(['school_name']).count()

#Display counts for 'student_name' column only
per_school_counts = school_groupby['student_name']

In [108]:
#Calculate the budget at each school
per_school_budget = school_summary['budget'].max()

#Calculate the capita at each school
per_school_capita = school_summary['budget'].max()/school_summary['size'].max()

In [109]:
#Calculate the average math score at each school
per_school_math = school_and_student.groupby('school_name')['math_score'].mean()

#Calculate the average reading score at each school
per_school_reading = school_and_student.groupby('school_name')['reading_score'].mean()

In [110]:
#Calculate the number of students passing math
students_passing_math = school_and_student[(school_and_student['math_score'] >= 70)]

#Calculate the number of students passing math at each school
school_students_passing_math = students_passing_math.groupby('school_name').math_score.count()

In [111]:
#Calculate the number of students passing reading
students_passing_reading = school_and_student[(school_and_student['reading_score'] >= 70)]

#Calculate the number of students passing reading at each school
school_students_passing_reading = students_passing_reading.groupby('school_name').reading_score.count()

In [113]:
#Determine the number of students passing both math and reading 
students_passing_math_and_reading = school_and_student[
    (school_and_student['reading_score'] >= 70) & (school_and_student['math_score'] >= 70)
]

#Calculate the number of students passing both math and reading at each school
school_students_passing_math_and_reading = students_passing_math_and_reading.groupby(['school_name']).size()

In [114]:
#Calculate the average number of students passing math at each school
per_school_passing_math = school_students_passing_math / per_school_counts * 100

#Calculate the average number of students passing reading at each school
per_school_passing_reading = school_students_passing_reading / per_school_counts * 100

#Calculate the average number of students passing both math and reading at each school
overall_passing_rate = school_students_passing_math_and_reading / per_school_counts * 100

In [115]:
#Summarize above calculations for each school in new DataFrame
per_school_summary = pd.DataFrame({
    'School Type': school_type,
    'Total Students': per_school_counts,
    'Total School Budget': per_school_budget,
    'Per Student Budget': per_school_capita,
    'Average Math Score': per_school_math,
    'Average Reading Score': per_school_reading,
    '% Passing Math': per_school_passing_math,
    '% Passing Reading': per_school_passing_reading,
    '% Overall Passing': overall_passing_rate
})

#Formatting
per_school_summary['Total School Budget'] = per_school_summary['Total School Budget'].map("${:,.2f}".format)
per_school_summary['Per Student Budget'] = per_school_summary['Per Student Budget'].map("${:,.2f}".format)

## Highest-Performing Schools (By % Overall Passing)

In [116]:
#Sort DataFrame by schools with the most students passing overall by percentage
top_schools = per_school_summary.sort_values(by=['% Overall Passing'], ascending=False)

## Bottom Performing Schools (By % Overall Passing)

In [117]:
#Sort DataFrame by schools with the least students passing overall by percentage
bottom_schools = per_school_summary.sort_values(by=['% Overall Passing'], ascending=True)

## Math Scores by Grade

In [118]:
#Sort students by grade
ninth_graders = school_and_student[(school_and_student['grade'] == '9th')]
tenth_graders = school_and_student[(school_and_student['grade'] == '10th')]
eleventh_graders = school_and_student[(school_and_student['grade'] == '11th')]
twelfth_graders = school_and_student[(school_and_student['grade'] == '12th')]

#Calculate average math score by grade
ninth_grade_math_scores = ninth_graders.groupby('school_name')['math_score'].mean()
tenth_grade_math_scores = tenth_graders.groupby('school_name')['math_score'].mean()
eleventh_grade_math_scores = eleventh_graders.groupby('school_name')['math_score'].mean()
twelfth_grade_math_scores = twelfth_graders.groupby('school_name')['math_score'].mean()

#Summarize avaerage math scores by grade in new DataFrame
math_scores_by_grade = pd.DataFrame({
    '9th Grade': ninth_grade_math_scores,
    '10th Grade': tenth_grade_math_scores,
    '11th Grade': eleventh_grade_math_scores,
    '12th Grade': twelfth_grade_math_scores
})

#Remove 'school_name' above schools
math_scores_by_grade.index.name = None

In [120]:
#Sort students by grade
ninth_graders = school_and_student[(school_and_student['grade'] == '9th')]
tenth_graders = school_and_student[(school_and_student['grade'] == '10th')]
eleventh_graders = school_and_student[(school_and_student['grade'] == '11th')]
twelfth_graders = school_and_student[(school_and_student['grade'] == '12th')]

#Calculate average reading score by grade
ninth_grade_reading_scores = ninth_graders.groupby('school_name')['reading_score'].mean()
tenth_grade_reading_scores = tenth_graders.groupby('school_name')['reading_score'].mean()
eleventh_grade_reading_scores = eleventh_graders.groupby('school_name')['reading_score'].mean()
twelfth_grade_reading_scores = twelfth_graders.groupby('school_name')['reading_score'].mean()

#Summarize average reading scores by grade in new DataFrame
reading_scores_by_grade = pd.DataFrame({
    '9th Grade': ninth_grade_reading_scores,
    '10th Grade': tenth_grade_reading_scores,
    '11th Grade': eleventh_grade_reading_scores,
    '12th Grade': twelfth_grade_reading_scores
})

#Remove 'school_name' above schools
reading_scores_by_grade.index.name = None

## Scores by School Spending

In [121]:
#Range to sort 'Per Student Budget' by
spending_bins = [0, 585, 630, 645, 680]

#Labels for bins
labels = ['<$585', '$585-630', '$630-645', '$645-680']

In [122]:
#Create new DataFrame variable identical to 'per_school_summary' DataFrame 
school_spending_df = per_school_summary.copy()

#Create new column called 'Spending Ranges (Per Student)' that will bin the 'Per Student Budget' column by above ranges
school_spending_df['Spending Ranges (Per Student)'] = pd.cut(per_school_capita, 
                                                             bins=spending_bins, 
                                                             right=True, 
                                                             labels=labels)

In [123]:
#Calculate the average math score by spending ranges per student
spending_math_scores = school_spending_df.groupby(['Spending Ranges (Per Student)'])['Average Math Score'].mean()

#Calculate the average reading score by spending ranges per student
spending_reading_scores = school_spending_df.groupby(['Spending Ranges (Per Student)'])['Average Reading Score'].mean()

#Calculate the percent of students passing math by spending ranges per student
spending_passing_math = school_spending_df.groupby(['Spending Ranges (Per Student)'])['% Passing Math'].mean()

#Calculate the percent of students passing reading by spending ranges per student
spending_passing_reading = school_spending_df.groupby(['Spending Ranges (Per Student)'])['% Passing Reading'].mean()

#Calculate the percent of students passing both math and reading by spending ranges per student
overall_passing_spending = school_spending_df.groupby(['Spending Ranges (Per Student)'])['% Overall Passing'].mean()

In [124]:
#Assemble scores by spending into DataFrame 
spending_summary = pd.DataFrame({
    'Average Math Score': spending_math_scores,
    'Average Reading Score': spending_reading_scores,
    '% Passing Math': spending_passing_math,
    '% Passing Reading': spending_passing_reading,
    '% Overall Passing': overall_passing_spending
})

## Scores by School Size

In [125]:
#Range to sort 'per_school_counts' by
size_bins = [0, 1000, 2000, 5000]

#Labels for bins
labels = ['Small (<1000)', 'Medium (1000-2000)', 'Large (2000-5000)']

In [126]:
#Create new column called 'School Size' that will bin 'per_school_counts' by the above ranges
per_school_summary['School Size'] = pd.cut(per_school_counts, 
                                           bins=size_bins, 
                                           right=True, 
                                           labels=labels)

In [127]:
#Calculate the average math score by school size
size_math_scores = per_school_summary.groupby(['School Size'])['Average Math Score'].mean()

#Calculate the average reading score by school size
size_reading_scores = per_school_summary.groupby(['School Size'])['Average Reading Score'].mean()

#Calculate the percent of students passing math by score by school size
size_passing_math = per_school_summary.groupby(['School Size'])['% Passing Math'].mean()

#Calculate the percent of students passing reading by score by school size
size_passing_reading = per_school_summary.groupby(['School Size'])['% Passing Reading'].mean()

#Calculate the percent of students passing both math and reading by score by school size
size_overall_passing = per_school_summary.groupby(['School Size'])['% Overall Passing'].mean()

In [128]:
#Assemble scores by school size into DataFrame 
size_summary = pd.DataFrame({
    'Average Math Score': size_math_scores,
    'Average Reading Score': size_reading_scores,
    '% Passing Math': size_passing_math,
    '% Passing Reading': size_passing_reading,
    '% Overall Passing': size_overall_passing
})

## Scores by School Type

In [129]:
#Calculate the average math score by school type
average_math_score_by_type = per_school_summary.groupby(['School Type'])['Average Math Score'].mean()

#Calculate the average reading score by school type
average_reading_score_by_type = per_school_summary.groupby(['School Type'])['Average Reading Score'].mean()

#Calculate the percent of student passing math by school type
average_percent_passing_math_by_type = per_school_summary.groupby(['School Type'])['% Passing Math'].mean()

#Calculate the percent of student passing reading by school type
average_percent_passing_reading_by_type = per_school_summary.groupby(['School Type'])['% Passing Reading'].mean()

#Calculate the percent of student passing both math and reading by school type
average_percent_overall_passing_by_type = per_school_summary.groupby(['School Type'])['% Overall Passing'].mean()

In [130]:
#Assemble scores by school type into DataFrame 
type_summary = pd.DataFrame({
    'Average Math Score': average_math_score_by_type,
    'Average Reading Score': average_reading_score_by_type,
    '% Passing Math': average_percent_passing_math_by_type,
    '% Passing Reading': average_percent_passing_reading_by_type,
    '% Overall Passing': average_percent_overall_passing_by_type
})